# Importing Libraries

In [3]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import gensim
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
 
warnings.filterwarnings(action='ignore')

# Processing Data

In [11]:
fake_news = pd.read_csv("Fake.csv")
true_news = pd.read_csv("True.csv")
# add labels
fake_news["label"] = "Fake"
true_news["label"] = "True"

# combine the two separate dataframes
corpus = pd.concat([fake_news, true_news], ignore_index = True)["text"]
tokenized_corpus = [simple_preprocess(sentence) for sentence in corpus]

In [12]:
corpus.head()

0    Donald Trump just couldn t wish all Americans ...
1    House Intelligence Committee Chairman Devin Nu...
2    On Friday, it was revealed that former Milwauk...
3    On Christmas day, Donald Trump announced that ...
4    Pope Francis used his annual Christmas Day mes...
Name: text, dtype: object

# CBOW and Skip-gram

In [15]:
model_cbow = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4, sg=0)
model_sg = Word2Vec(sentences=tokenized_corpus, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Parameters:
# - vector_size: Dimensionality of word embeddings
# - window: Context window size
# - min_count: Ignores words appearing less than min_count times
# - workers: Number of CPU cores to use
# - sg: 0 for CBOW (Continuous Bag of Words), 1 for Skip-gram

# Save and Load Model

In [19]:
model_cbow.save("word2vec_cbow.model")
model_sg.save("word2vec_sg.model")

# Load the model
# loaded_model = Word2Vec.load("word2vec.model")